# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

In [1]:
! pip install -U langchain

   ---------------------------------------- 0.0/990.0 kB ? eta -:--:--
   - -------------------------------------- 30.7/990.0 kB 1.3 MB/s eta 0:00:01
   -- ------------------------------------ 61.4/990.0 kB 812.7 kB/s eta 0:00:02
   ---- --------------------------------- 112.6/990.0 kB 930.9 kB/s eta 0:00:01
   ------ --------------------------------- 163.8/990.0 kB 1.1 MB/s eta 0:00:01
   --------- ------------------------------ 225.3/990.0 kB 1.1 MB/s eta 0:00:01
   --------- ------------------------------ 225.3/990.0 kB 1.1 MB/s eta 0:00:01
   --------- ------------------------------ 225.3/990.0 kB 1.1 MB/s eta 0:00:01
   ----------- -------------------------- 297.0/990.0 kB 916.0 kB/s eta 0:00:01
   ------------ ------------------------- 327.7/990.0 kB 846.5 kB/s eta 0:00:01
   ------------ ------------------------- 327.7/990.0 kB 846.5 kB/s eta 0:00:01
   ------------ ------------------------- 327.7/990.0 kB 846.5 kB/s eta 0:00:01
   ------------ ------------------------- 327.7/99

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.1.7 requires openai<2.0.0,>=1.24.0, but you have openai 1.14.2 which is incompatible.


## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

## Router Chain: 实现条件判断的大模型调用


这段代码构建了一个可定制的链路系统，用户可以提供不同的输入提示，并根据这些提示获取适当的响应。

主要逻辑：从`prompt_infos`创建多个`LLMChain`对象，并将它们保存在一个字典中，然后创建一个默认的`ConversationChain`，最后创建一个带有路由功能的`MultiPromptChain`。

![](../images/router_chain.png)

In [2]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [6]:
physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""


math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

biotechnology_template = """你是一位很棒的生物学家。你擅长回答生物方面的问题。
之所以如此出色，是因为你能够将晦涩难懂的生物知识用通俗易懂讲给不具备生物知识的人听，
即举个例子去阐述原理。

这是一个问题：
{input}"""

cs_template = """你是一位很棒的计算机教授。你擅长回答计算机原理问题以及查找bug。
之所以如此出色，是因为你能够找出不同技术的共通之处，
先回答这个技术的相关问题，然后再将与他相关的内容也整理出来，帮助学生触类旁通。

这是一个问题：
{input}"""

chinese_language_template = """你是一位很棒的汉语言老师。你擅长回答有关中文的问题。
之所以如此出色，是因为你对中华文化、各类典故了解很深，回答问题的时候总能引经据典。

这是一个问题：
{input}"""

In [7]:
prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
    {
        "name": "生物",
        "description": "适用于回答生物问题",
        "prompt_template": biotechnology_template,
    },
    {
        "name": "计算机",
        "description": "适用于回答计算机问题",
        "prompt_template": cs_template,
    },
        {
        "name": "汉语言",
        "description": "适用于回答汉语言问题",
        "prompt_template": chinese_language_template,
    }
]

In [24]:
import os
 
BASE_URL = os.environ.get("https://api.xiaoai.plus/v1")

API_KEY = os.environ.get("OPENAI_API_KEY")
 
llm = OpenAI(model_name="gpt-3.5-turbo-instruct", base_url=BASE_URL, api_key=API_KEY)
 

In [25]:
# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

In [26]:
type(default_chain)

langchain.chains.conversation.base.ConversationChain

### 使用 LLMRouterChain 实现条件判断调用

这段代码定义了一个chain对象（LLMRouterChain），该对象首先使用router_chain来决定哪个destination_chain应该被执行，如果没有合适的目标链，则默认使用default_chain。

In [27]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [28]:
# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [29]:
print(destinations)

['物理: 适用于回答物理问题', '数学: 适用于回答数学问题', '生物: 适用于回答生物问题', '计算机: 适用于回答计算机问题', '汉语言: 适用于回答汉语言问题']


In [30]:
print(destinations_str)

物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机问题
汉语言: 适用于回答汉语言问题


In [31]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [32]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机问题
汉语言: 适用于回答汉语言问题

<< INPUT >>
{input}

<< OU

In [18]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [33]:
print(chain.invoke("细胞包含哪些结构?"))

{'input': '细胞包含哪些结构?', 'text': '\n\n感谢您的提问！细胞是生物体的基本结构和功能单位，包含以下几个主要结构：\n\n1. 细胞膜：包裹在细胞外部，起着筛选物质进出细胞的作用。\n\n2. 细胞质：细胞膜内部的液体，包含各种细胞器和细胞器官。\n\n3. 细胞核：位于细胞质中央，包含遗传物质DNA，并控制细胞的生长和分裂。\n\n4. 线粒体：细胞内的能量生产中心，负责细胞内的呼吸作用，产生能量供细胞使用。\n\n5. 液泡：细胞内的小囊泡，负责储存物质和调节细胞内的压力。\n\n6'}


In [34]:
print(
    chain.invoke(
        "redis的集群模式有哪些？"
    )
)

{'input': 'redis的集群模式有哪些？', 'text': '\nRedis的集群模式有以下几种：\n\n1. 主从复制模式：将数据复制到多个从节点，实现读写分离，提高读取性能和数据的可靠性。\n2. Sentinel模式：在主从复制模式的基础上引入Sentinel节点，用于监控主节点的健康状态，并在主节点宕机时自动将从节点提升为主节点。\n3. Cluster模式：采用无中心节点的分布式架构，将数据分散到多个节点上，实现数据的高可用和扩展性。\n4. Proxy模式：通过中间件代理服务器，将多个Redis节点组合成一个逻辑集群，对外提供统一的访问接口，实现负载均衡和高可用性。'}


In [35]:
router_chain.verbose = True

In [36]:
print(chain.invoke("汉字 家 的来源？"))

{'input': '汉字 家 的来源？', 'text': '\n\n汉字 "家" 的来源可以追溯到甲骨文时期。在甲骨文中，"家"的形状是一个象形字，像一个小房子，表示住宅的意思。后来，随着社会的发展，"家"的含义也扩大，不仅仅指住宅，还包括家庭、家人、家业等等。在现代汉语中，"家"的含义更加广泛，可以指代家庭、家乡、家具、家电等等。'}


### Homework

#### 扩展 Demo：实现生物、计算机和汉语言文学老师 PromptTemplates 及对应 Chains